## Import Dependencies

In [1]:
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

## Retrieve game schedules for Houston Astros collected from baseball-reference.com
Determine which seasons schedules to pull by listing the season year in the "years" list. 

In [2]:
# create a base url and list of years which you would like to pull game schedules for
base_url = 'https://www.baseball-reference.com/teams/HOU/'
years = ['2017', '2018', '2019']

In [3]:
# create a list of dataframes
# loop through years to create target url, scrape tabular data from webpage using read_html pandas function, 
# save first table instance and add/drop columns to dataframe and append dataframe to dataframes list
dfs = []
for year in years:
    url = base_url + year + '-schedule-scores.shtml'
    df = pd.read_html(url)
    df = df[0]
    df['year']=year
    df = df.set_index("Gm#")
    df = df.drop("Gm#", axis=0)
    df = df.drop("Unnamed: 2", axis=1)
    df = df.drop("Save", axis=1)
    df = df.drop("Orig. Scheduled", axis=1)
    df = df.rename(columns={"Unnamed: 4":"At"})
    dfs.append(df)

In [4]:
# We now have a list of dataframes for each season in the "years" list. 
# We will now merge the dataframes in this list into one combined dataframe

# create a list for the number of dfs minus one
no_of_dfs=np.arange(len(dfs)-1)

# set first df in list to a variable
comb_df=dfs[0]

# append remaining dfs in list to first df
for x in no_of_dfs:
    comb_df=comb_df.append(dfs[x+1])
    

In [5]:
# clean combined dataframe by assigning "away" or "home" to "At" column to make df clearer
comb_df['At']=comb_df['At'].replace({"@":"away"})
comb_df['At']=comb_df['At'].fillna("home")
comb_df.head(10)

,Date,Tm,At,Opp,W/L,R,RA,Inn,W-L,Rank,GB,Win,Loss,Time,D/N,Attendance,Streak,year
Gm#,,,,,,,,,,,,,,,,,,
1,"Monday, Apr 3",HOU,home,SEA,W,3,0,NaN,1-0,1,Tied,Keuchel,Hernandez,2:29,N,41678,+,2017
2,"Tuesday, Apr 4",HOU,home,SEA,W,2,1,NaN,2-0,1,up 1.0,McCullers,Iwakuma,2:41,N,21406,++,2017
3,"Wednesday, Apr 5",HOU,home,SEA,W-wo,5,3,13,3-0,1,up 1.0,Peacock,De Jong,4:25,N,20303,+++,2017
4,"Thursday, Apr 6",HOU,home,SEA,L,2,4,NaN,3-1,1,up 1.0,Altavilla,Giles,2:57,N,18362,-,2017
5,"Friday, Apr 7",HOU,home,KCR,L,1,5,NaN,3-2,1,Tied,Vargas,Fiers,3:12,N,30491,--,2017
6,"Saturday, Apr 8",HOU,home,KCR,L,3,7,NaN,3-3,2,1.0,Duffy,Gregerson,2:42,N,35373,---,2017
7,"Sunday, Apr 9",HOU,home,KCR,W-wo,5,4,12,4-3,2,1.0,Devenski,Strahm,3:49,D,32411,+,2017
8,"Monday, Apr 10",HOU,away,SEA,L,0,6,NaN,4-4,2,1.5,Paxton,Morton,2:56,D,44856,-,2017
9,"Tuesday, Apr 11",HOU,away,SEA,W,7,5,NaN,5-4,2,1.5,Musgrove,Miranda,3:29,N,18527,+,2017


In [6]:
# add an empty column called location
comb_df['location_team_abbrv']=''

# identify the location of the game by assigning the team abbreviation based on whether the game was home or away 
for index, row in comb_df.iterrows():
    if row['At']=="away":
        row["location_team_abbrv"]=row["Opp"]
    else:
        row["location_team_abbrv"]=row["Tm"]
comb_df.head(15)

,Date,Tm,At,Opp,W/L,R,RA,Inn,W-L,Rank,GB,Win,Loss,Time,D/N,Attendance,Streak,year,location_team_abbrv
Gm#,,,,,,,,,,,,,,,,,,,
1,"Monday, Apr 3",HOU,home,SEA,W,3,0,NaN,1-0,1,Tied,Keuchel,Hernandez,2:29,N,41678,+,2017,HOU
2,"Tuesday, Apr 4",HOU,home,SEA,W,2,1,NaN,2-0,1,up 1.0,McCullers,Iwakuma,2:41,N,21406,++,2017,HOU
3,"Wednesday, Apr 5",HOU,home,SEA,W-wo,5,3,13,3-0,1,up 1.0,Peacock,De Jong,4:25,N,20303,+++,2017,HOU
4,"Thursday, Apr 6",HOU,home,SEA,L,2,4,NaN,3-1,1,up 1.0,Altavilla,Giles,2:57,N,18362,-,2017,HOU
5,"Friday, Apr 7",HOU,home,KCR,L,1,5,NaN,3-2,1,Tied,Vargas,Fiers,3:12,N,30491,--,2017,HOU
6,"Saturday, Apr 8",HOU,home,KCR,L,3,7,NaN,3-3,2,1.0,Duffy,Gregerson,2:42,N,35373,---,2017,HOU
7,"Sunday, Apr 9",HOU,home,KCR,W-wo,5,4,12,4-3,2,1.0,Devenski,Strahm,3:49,D,32411,+,2017,HOU
8,"Monday, Apr 10",HOU,away,SEA,L,0,6,NaN,4-4,2,1.5,Paxton,Morton,2:56,D,44856,-,2017,SEA
9,"Tuesday, Apr 11",HOU,away,SEA,W,7,5,NaN,5-4,2,1.5,Musgrove,Miranda,3:29,N,18527,+,2017,SEA


In [7]:
# rename column headers appropriately
comb_df=comb_df.rename(columns={"Date":"game_date","Tm":"team","Opp":"opp","W/L":"win/loss","R":"runs_scored","RA":"runs_allowed","Inn":"innings","W-L":"win_loss_rec","Rank":"rank","GB":"games_back","Win":"win","Loss":"loss","Time":"game_time","D/N":"day/night","Attendance":"attendance","Streak":"streak","year":"game_year"})
comb_df= comb_df.reset_index()
comb_df['id'] = comb_df.index
comb_df.head()

,Gm#,game_date,team,At,opp,win/loss,runs_scored,runs_allowed,innings,win_loss_rec,...,games_back,win,loss,game_time,day/night,attendance,streak,game_year,location_team_abbrv,id
0,1,"Monday, Apr 3",HOU,home,SEA,W,3,0,NaN,1-0,...,Tied,Keuchel,Hernandez,2:29,N,41678,+,2017,HOU,0
1,2,"Tuesday, Apr 4",HOU,home,SEA,W,2,1,NaN,2-0,...,up 1.0,McCullers,Iwakuma,2:41,N,21406,++,2017,HOU,1
2,3,"Wednesday, Apr 5",HOU,home,SEA,W-wo,5,3,13,3-0,...,up 1.0,Peacock,De Jong,4:25,N,20303,+++,2017,HOU,2
3,4,"Thursday, Apr 6",HOU,home,SEA,L,2,4,NaN,3-1,...,up 1.0,Altavilla,Giles,2:57,N,18362,-,2017,HOU,3
4,5,"Friday, Apr 7",HOU,home,KCR,L,1,5,NaN,3-2,...,Tied,Vargas,Fiers,3:12,N,30491,--,2017,HOU,4


## Retrieve Ball Park Data from Wiki

In [8]:
# scrape tabular data from webpage using read_html pandas function, save first table instance and drop columns not needed
url_wiki = 'https://en.wikipedia.org/wiki/List_of_current_Major_League_Baseball_stadiums'
wiki_df=pd.read_html(url_wiki)
wiki_df=wiki_df[1]
wiki_df = wiki_df.drop("Image", axis=1)
wiki_df

,Name,Capacity,Location,Surface,Team,Opened,Distance to center field,Type,Roof type
0,Angel Stadium,"45,517[1]","Anaheim, California",Grass,Los Angeles Angels,1966,396 feet (121 m),Modern,Open
1,Busch Stadium,"45,494[2]","St. Louis, Missouri",Grass,St. Louis Cardinals,2006,400 feet (122 m),Retro-classic,Open
2,Chase Field,"48,686[3]","Phoenix, Arizona",Artificial turf,Arizona Diamondbacks,1998,407 feet (124 m),Retro-modern,Retractable
3,Citi Field,"41,922[4]","Queens, New York",Grass,New York Mets,2009,408 feet (124 m),Retro-classic,Open
4,Citizens Bank Park,"42,792[5]","Philadelphia, Pennsylvania",Grass,Philadelphia Phillies,2004,401 feet (122 m),Retro-classic,Open
5,Comerica Park,"41,083[6]","Detroit, Michigan",Grass,Detroit Tigers,2000,420 feet (128 m),Retro-classic,Open
6,Coors Field,"50,445[7]","Denver, Colorado",Grass,Colorado Rockies,1995,415 feet (126 m),Retro-classic,Open
7,Dodger Stadium,"56,000[8]","Los Angeles, California",Grass,Los Angeles Dodgers[nb 2],1962,395 feet (120 m),Modern,Open
8,Fenway Park,"37,755[9]","Boston, Massachusetts",Grass,Boston Red Sox[nb 3],1912,390 feet (119 m),Jewel box,Open
9,Globe Life Park in Arlington,"48,114[10]","Arlington, Texas",Grass,Texas Rangers,1994,400 feet (122 m),Retro-classic,Open


In [9]:
# clean dataframe and data inside dataframe to make it look uniform
wiki_df['Capacity']=wiki_df['Capacity'].apply(lambda x : x.split("[")[0])
wiki_df['Team']=wiki_df['Team'].apply(lambda x : x.split("[")[0])
wiki_df['Opened']=wiki_df['Opened'].apply(lambda x : x.split("[")[0])
wiki_df['Distance to center field']=wiki_df['Distance to center field'].apply(lambda x : x.split("[")[0])
wiki_df['Type']=wiki_df['Type'].apply(lambda x : x.split("[")[0])
wiki_df = wiki_df.rename(columns={"Name":"bp_name","Capacity":"bp_capacity","Location":"bp_location","Surface":"bp_surface","Team":"team_name_bp","Opened":"bp_opened","Distance to center field":"bp_dst_cf","Type":"bp_type","Roof type":"bp_roof_type"})
wiki_df

,bp_name,bp_capacity,bp_location,bp_surface,team_name_bp,bp_opened,bp_dst_cf,bp_type,bp_roof_type
0,Angel Stadium,"45,517","Anaheim, California",Grass,Los Angeles Angels,1966,396 feet (121 m),Modern,Open
1,Busch Stadium,"45,494","St. Louis, Missouri",Grass,St. Louis Cardinals,2006,400 feet (122 m),Retro-classic,Open
2,Chase Field,"48,686","Phoenix, Arizona",Artificial turf,Arizona Diamondbacks,1998,407 feet (124 m),Retro-modern,Retractable
3,Citi Field,"41,922","Queens, New York",Grass,New York Mets,2009,408 feet (124 m),Retro-classic,Open
4,Citizens Bank Park,"42,792","Philadelphia, Pennsylvania",Grass,Philadelphia Phillies,2004,401 feet (122 m),Retro-classic,Open
5,Comerica Park,"41,083","Detroit, Michigan",Grass,Detroit Tigers,2000,420 feet (128 m),Retro-classic,Open
6,Coors Field,"50,445","Denver, Colorado",Grass,Colorado Rockies,1995,415 feet (126 m),Retro-classic,Open
7,Dodger Stadium,"56,000","Los Angeles, California",Grass,Los Angeles Dodgers,1962,395 feet (120 m),Modern,Open
8,Fenway Park,"37,755","Boston, Massachusetts",Grass,Boston Red Sox,1912,390 feet (119 m),Jewel box,Open
9,Globe Life Park in Arlington,"48,114","Arlington, Texas",Grass,Texas Rangers,1994,400 feet (122 m),Retro-classic,Open


## Retrieve MLB Franchise Data Including Team Name, Team ID & Franchise ID from baseball-reference.com

In [10]:
# scrape tabular data from webpage using read_html pandas function, save first table instance, drop unnecessary rows/columns, rename columns
url_teams = 'https://www.baseball-reference.com/about/team_IDs.shtml'
franchise_df=pd.read_html(url_teams)
franchise_df=franchise_df[0]
franchise_df = franchise_df.drop(0, axis=0)
franchise_df = franchise_df.rename(columns={0:"franchise_id", 1:"team_id", 2:"team_name_fran", 3:"fran_first_year", 4:"fran_last_year"})
franchise_df.head()

,franchise_id,team_id,team_name_fran,fran_first_year,fran_last_year
1,ALT,ALT,Altoona Mountain City,1884,1884
2,ANA,ANA,Los Angeles Angels of Anaheim,1997,Present
3,ANA,CAL,California Angels,1965,1996
4,ANA,LAA,Los Angeles Angels,1961,1964
5,ARI,ARI,Arizona Diamondbacks,1998,Present


In [11]:
# create dataframe for current teams 
team_fran_df=franchise_df.loc[franchise_df['fran_last_year']=="Present",:]
team_fran_df=team_fran_df.reset_index(drop=True)
team_fran_df

,franchise_id,team_id,team_name_fran,fran_first_year,fran_last_year
0,ANA,ANA,Los Angeles Angels of Anaheim,1997,Present
1,ARI,ARI,Arizona Diamondbacks,1998,Present
2,ATL,ATL,Atlanta Braves,1966,Present
3,BAL,BAL,Baltimore Orioles,1954,Present
4,BOS,BOS,Boston Red Sox,1901,Present
5,CHC,CHC,Chicago Cubs,1876,Present
6,CHW,CHW,Chicago White Sox,1901,Present
7,CIN,CIN,Cincinnati Reds,1882,Present
8,CLE,CLE,Cleveland Indians,1901,Present
9,COL,COL,Colorado Rockies,1993,Present


## Retrieve Data from name_abbrv.csv file 
The team name and abbreviations vary slightly for each dataframe above. To make sure my data acurately merges, 
I created a csv file with the name reference for each team specific to the dataframes above.

In [24]:
data_file=os.path.join('name_abbrv.csv')
name_abbrv=pd.read_csv(data_file)
name_abbrv['id'] = name_abbrv.index
name_abbrv

,team_name_fran,team_name_bp,location_team_abbrv,id
0,Arizona Diamondbacks,Arizona Diamondbacks,ARI,0
1,Atlanta Braves,Atlanta Braves,ATL,1
2,Baltimore Orioles,Baltimore Orioles,BAL,2
3,Boston Red Sox,Boston Red Sox,BOS,3
4,Chicago Cubs,Chicago Cubs,CHC,4
5,Chicago White Sox,Chicago White Sox,CHW,5
6,Cincinnati Reds,Cincinnati Reds,CIN,6
7,Cleveland Indians,Cleveland Indians,CLE,7
8,Colorado Rockies,Colorado Rockies,COL,8
9,Detroit Tigers,Detroit Tigers,DET,9


### Merge Dataframes

In [13]:
park_name=pd.merge(wiki_df,name_abbrv,on="team_name_bp",how='outer')
park_name

,bp_name,bp_capacity,bp_location,bp_surface,team_name_bp,bp_opened,bp_dst_cf,bp_type,bp_roof_type,team_name_fran,location_team_abbrv
0,Angel Stadium,"45,517","Anaheim, California",Grass,Los Angeles Angels,1966,396 feet (121 m),Modern,Open,Los Angeles Angels of Anaheim,LAA
1,Busch Stadium,"45,494","St. Louis, Missouri",Grass,St. Louis Cardinals,2006,400 feet (122 m),Retro-classic,Open,St. Louis Cardinals,STL
2,Chase Field,"48,686","Phoenix, Arizona",Artificial turf,Arizona Diamondbacks,1998,407 feet (124 m),Retro-modern,Retractable,Arizona Diamondbacks,ARI
3,Citi Field,"41,922","Queens, New York",Grass,New York Mets,2009,408 feet (124 m),Retro-classic,Open,New York Mets,NYM
4,Citizens Bank Park,"42,792","Philadelphia, Pennsylvania",Grass,Philadelphia Phillies,2004,401 feet (122 m),Retro-classic,Open,Philadelphia Phillies,PHI
5,Comerica Park,"41,083","Detroit, Michigan",Grass,Detroit Tigers,2000,420 feet (128 m),Retro-classic,Open,Detroit Tigers,DET
6,Coors Field,"50,445","Denver, Colorado",Grass,Colorado Rockies,1995,415 feet (126 m),Retro-classic,Open,Colorado Rockies,COL
7,Dodger Stadium,"56,000","Los Angeles, California",Grass,Los Angeles Dodgers,1962,395 feet (120 m),Modern,Open,Los Angeles Dodgers,LAD
8,Fenway Park,"37,755","Boston, Massachusetts",Grass,Boston Red Sox,1912,390 feet (119 m),Jewel box,Open,Boston Red Sox,BOS
9,Globe Life Park in Arlington,"48,114","Arlington, Texas",Grass,Texas Rangers,1994,400 feet (122 m),Retro-classic,Open,Texas Rangers,TEX


In [14]:
park_fran_name=pd.merge(park_name,team_fran_df,on='team_name_fran',how='outer')
park_fran_name

,bp_name,bp_capacity,bp_location,bp_surface,team_name_bp,bp_opened,bp_dst_cf,bp_type,bp_roof_type,team_name_fran,location_team_abbrv,franchise_id,team_id,fran_first_year,fran_last_year
0,Angel Stadium,"45,517","Anaheim, California",Grass,Los Angeles Angels,1966,396 feet (121 m),Modern,Open,Los Angeles Angels of Anaheim,LAA,ANA,ANA,1997,Present
1,Busch Stadium,"45,494","St. Louis, Missouri",Grass,St. Louis Cardinals,2006,400 feet (122 m),Retro-classic,Open,St. Louis Cardinals,STL,STL,STL,1882,Present
2,Chase Field,"48,686","Phoenix, Arizona",Artificial turf,Arizona Diamondbacks,1998,407 feet (124 m),Retro-modern,Retractable,Arizona Diamondbacks,ARI,ARI,ARI,1998,Present
3,Citi Field,"41,922","Queens, New York",Grass,New York Mets,2009,408 feet (124 m),Retro-classic,Open,New York Mets,NYM,NYM,NYM,1962,Present
4,Citizens Bank Park,"42,792","Philadelphia, Pennsylvania",Grass,Philadelphia Phillies,2004,401 feet (122 m),Retro-classic,Open,Philadelphia Phillies,PHI,PHI,PHI,1883,Present
5,Comerica Park,"41,083","Detroit, Michigan",Grass,Detroit Tigers,2000,420 feet (128 m),Retro-classic,Open,Detroit Tigers,DET,DET,DET,1901,Present
6,Coors Field,"50,445","Denver, Colorado",Grass,Colorado Rockies,1995,415 feet (126 m),Retro-classic,Open,Colorado Rockies,COL,COL,COL,1993,Present
7,Dodger Stadium,"56,000","Los Angeles, California",Grass,Los Angeles Dodgers,1962,395 feet (120 m),Modern,Open,Los Angeles Dodgers,LAD,LAD,LAD,1958,Present
8,Fenway Park,"37,755","Boston, Massachusetts",Grass,Boston Red Sox,1912,390 feet (119 m),Jewel box,Open,Boston Red Sox,BOS,BOS,BOS,1901,Present
9,Globe Life Park in Arlington,"48,114","Arlington, Texas",Grass,Texas Rangers,1994,400 feet (122 m),Retro-classic,Open,Texas Rangers,TEX,TEX,TEX,1972,Present


In [15]:
master_df=pd.merge(comb_df,park_fran_name,on="location_team_abbrv",how="left")
master_df.head(10)

,Gm#,game_date,team,At,opp,win/loss,runs_scored,runs_allowed,innings,win_loss_rec,...,team_name_bp,bp_opened,bp_dst_cf,bp_type,bp_roof_type,team_name_fran,franchise_id,team_id,fran_first_year,fran_last_year
0,1,"Monday, Apr 3",HOU,home,SEA,W,3,0,NaN,1-0,...,Houston Astros,2000,409 feet (125 m),Retro-modern,Retractable,Houston Astros,HOU,HOU,1962,Present
1,2,"Tuesday, Apr 4",HOU,home,SEA,W,2,1,NaN,2-0,...,Houston Astros,2000,409 feet (125 m),Retro-modern,Retractable,Houston Astros,HOU,HOU,1962,Present
2,3,"Wednesday, Apr 5",HOU,home,SEA,W-wo,5,3,13,3-0,...,Houston Astros,2000,409 feet (125 m),Retro-modern,Retractable,Houston Astros,HOU,HOU,1962,Present
3,4,"Thursday, Apr 6",HOU,home,SEA,L,2,4,NaN,3-1,...,Houston Astros,2000,409 feet (125 m),Retro-modern,Retractable,Houston Astros,HOU,HOU,1962,Present
4,5,"Friday, Apr 7",HOU,home,KCR,L,1,5,NaN,3-2,...,Houston Astros,2000,409 feet (125 m),Retro-modern,Retractable,Houston Astros,HOU,HOU,1962,Present
5,6,"Saturday, Apr 8",HOU,home,KCR,L,3,7,NaN,3-3,...,Houston Astros,2000,409 feet (125 m),Retro-modern,Retractable,Houston Astros,HOU,HOU,1962,Present
6,7,"Sunday, Apr 9",HOU,home,KCR,W-wo,5,4,12,4-3,...,Houston Astros,2000,409 feet (125 m),Retro-modern,Retractable,Houston Astros,HOU,HOU,1962,Present
7,8,"Monday, Apr 10",HOU,away,SEA,L,0,6,NaN,4-4,...,Seattle Mariners,1999,401 feet (122 m),Retro-modern,Retractable,Seattle Mariners,SEA,SEA,1977,Present
8,9,"Tuesday, Apr 11",HOU,away,SEA,W,7,5,NaN,5-4,...,Seattle Mariners,1999,401 feet (122 m),Retro-modern,Retractable,Seattle Mariners,SEA,SEA,1977,Present
9,10,"Wednesday, Apr 12",HOU,away,SEA,W,10,5,NaN,6-4,...,Seattle Mariners,1999,401 feet (122 m),Retro-modern,Retractable,Seattle Mariners,SEA,SEA,1977,Present


## Load dataframes into Postgres "etl_project" database

In [16]:
db_user='postgres'
db_password='postgres'
db_host='localhost'
db_port=5432
db_name='etl_project'
engine = create_engine(f"postgres://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

In [25]:
name_abbrv.to_sql('reference', engine, if_exists='replace')
with engine.connect() as con:
    con.execute('ALTER TABLE reference ADD PRIMARY KEY ("id");')

In [18]:
wiki_df.to_sql('parks', engine, if_exists='replace')
with engine.connect() as con:
    con.execute('ALTER TABLE parks ADD PRIMARY KEY ("team_name_bp");')

In [19]:
team_fran_df.to_sql('franchise', engine, if_exists='replace')
with engine.connect() as con:
    con.execute('ALTER TABLE franchise ADD PRIMARY KEY ("team_name_fran");')

In [27]:
comb_df.to_sql('schedule', engine, if_exists='replace')
with engine.connect() as con:
    con.execute('ALTER TABLE schedule ADD PRIMARY KEY ("id");')